In [1]:
#!/usr/bin/env python
from __future__ import print_function

import numpy as np
import pandas as pd
import h5py
import re

In [19]:
from datetime import datetime

f = h5py.File("/Users/rhf/MANDI_5800_event.nxs", "r")



event_banks = [item for item in f["entry"].values() if isinstance(item, h5py.Group) and item.name.endswith("events")]

df = pd.DataFrame()


In [20]:
for bank in event_banks[:2]: # For debug use event_banks[-1:]
    
    #
    # index is event id, value is pixel_id
    event_pixel_id = bank["event_id"].value
    
    #
    # create tmp df with the pixel_id inside
    df_tmp = pd.DataFrame(
        data = event_pixel_id,
        columns = ['pixel_id']
    )
    
    #
    # Get the Pixels position 256x256 = 65536
    pixel_ids = bank["pixel_id"].value
    # flatten pixel_ids and get respective indices
    XX,YY = np.meshgrid(np.arange(pixel_ids.shape[1]),np.arange(pixel_ids.shape[0]))
    # 2d array of the format: pixel_id, i, j
    pixel_id_and_indices = np.vstack((pixel_ids.ravel(),XX.ravel(),YY.ravel())).T
    
    # tmp df for detector info
    df_tmp2 = pd.DataFrame(
        data=pixel_id_and_indices,
        columns=['pixel_id', 'i', 'j'],
    )
    # Left join
    df_tmp = pd.merge(left=df_tmp,right=df_tmp2, how='left', left_on='pixel_id', right_on='pixel_id')
    # remove the pixel_id (we don't need it amy more)
    df_tmp = df_tmp.drop('pixel_id', 1)

    #
    # index is event id, value time
    event_time_offset = bank["event_time_offset"].value
    df_tmp['time_offset'] = event_time_offset





    # Length pulses != events
    #
    # index is pulse id, value is event_id
    pulse_event_index = bank["event_index"].value
    pulse_event_index_step = np.diff(pulse_event_index) # difference between the current and previous
    pulse_event_index_step = np.insert(pulse_event_index_step,0,0) # Insert 0 in the beggining
    #
    # index is pulse id, value pulse 0 time
    pulse_time_zero = bank["event_time_zero"].value
    pulse_time_zero_with_event = pulse_time_zero[pulse_event_index_step>0] # only where there is a count
    pulse_event_index_with_event = pulse_event_index[pulse_event_index_step>0]

    df_tmp2 = pd.DataFrame(
        data= np.column_stack((pulse_event_index_with_event,pulse_time_zero_with_event)),
        columns=['event_id', 'time_zero'],
    )
    df_tmp2.event_id = df_tmp2.event_id.astype(np.int64)
    # Left join
    df_tmp = pd.merge(left=df_tmp,right=df_tmp2, how='left',  left_index = True, right_on='event_id')
    df_tmp = df_tmp.fillna(method='bfill') # fill nans with the value from the next cell



    #
    # get the bank number
    pattern = re.compile(r"/entry/bank(\d+)_events")
    match = pattern.match(bank.name)
    bank_number = int(match.group(1))

    #
    # Bank Number
    bank_number_array = np.array([bank_number]*len(df_tmp))
    df_tmp['bank_id'] = bank_number_array

    # Append to the final table
    df = df.append(df_tmp, ignore_index = True)

df


,i,j,time_offset,event_id,time_zero,bank_id
0,240,206,22161.199219,0,0.066666,11
1,175,131,8810.299805,1,0.066666,11
2,126,145,12683.000000,2,0.066666,11
3,105,233,14681.900391,3,0.066666,11
4,84,54,18583.199219,4,0.183330,11
5,244,216,19682.300781,5,0.283328,11
6,29,29,20272.300781,6,0.283328,11
7,31,107,20595.699219,7,0.283328,11
8,21,42,10271.000000,8,0.366660,11
9,85,88,18405.599609,9,0.399993,11


In [21]:
#
# Convert TOF number to time_delta
# SLOW!
df['time_offset'] = pd.to_timedelta(df['time_offset'],  unit="us")
df

,i,j,time_offset,event_id,time_zero,bank_id
0,240,206,00:00:00.022161,0,0.066666,11
1,175,131,00:00:00.008810,1,0.066666,11
2,126,145,00:00:00.012683,2,0.066666,11
3,105,233,00:00:00.014681,3,0.066666,11
4,84,54,00:00:00.018583,4,0.183330,11
5,244,216,00:00:00.019682,5,0.283328,11
6,29,29,00:00:00.020272,6,0.283328,11
7,31,107,00:00:00.020595,7,0.283328,11
8,21,42,00:00:00.010271,8,0.366660,11
9,85,88,00:00:00.018405,9,0.399993,11


In [22]:
# Just to make sure start and end times makes sense
start_date = np.datetime64(f['entry']['start_time'].value[0])
print(start_date)
end_time = np.datetime64(f['entry']['end_time'].value[0])
print(end_time)

# SLOW
df['time_zero'] = pd.to_timedelta(df.time_zero, unit='s') + start_date
df = df.set_index('time_zero')

# We don't need event id
df = df.drop('event_id', 1)
df

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  after removing the cwd from sys.path.


2016-07-15T06:50:28.825350666
2016-07-15T11:34:15.798428666


,i,j,time_offset,bank_id
time_zero,,,,
2016-07-15 06:50:28.892016666,240,206,00:00:00.022161,11
2016-07-15 06:50:28.892016666,175,131,00:00:00.008810,11
2016-07-15 06:50:28.892016666,126,145,00:00:00.012683,11
2016-07-15 06:50:28.892016666,105,233,00:00:00.014681,11
2016-07-15 06:50:29.008680666,84,54,00:00:00.018583,11
2016-07-15 06:50:29.108678666,244,216,00:00:00.019682,11
2016-07-15 06:50:29.108678666,29,29,00:00:00.020272,11
2016-07-15 06:50:29.108678666,31,107,00:00:00.020595,11
2016-07-15 06:50:29.192010666,21,42,00:00:00.010271,11


In [23]:
#
# Let's bin now
#

# TOF Range
df.time_offset.min(),df.time_offset.max()

(Timedelta('0 days 00:00:00.007048'), Timedelta('0 days 00:00:00.023714'))

In [24]:
# Select a date range
df.loc['2016-07-15 09:00:00.0':'2016-07-15 10:00:00.0']

,i,j,time_offset,bank_id
time_zero,,,,
2016-07-15 09:00:00.058021666,176,14,00:00:00.015168,11
2016-07-15 09:00:00.091353666,162,63,00:00:00.020588,11
2016-07-15 09:00:00.091353666,199,153,00:00:00.012092,11
2016-07-15 09:00:00.141353666,13,83,00:00:00.015559,11
2016-07-15 09:00:00.141353666,24,30,00:00:00.017491,11
2016-07-15 09:00:00.174685666,27,109,00:00:00.014980,11
2016-07-15 09:00:00.208018666,152,132,00:00:00.020009,11
2016-07-15 09:00:00.224684666,92,80,00:00:00.015028,11
2016-07-15 09:00:00.241351666,194,96,00:00:00.016793,11


In [28]:
# Let's see how many neutrons every pixel
df.groupby(["bank_id","i","j"]).count()

time_offset
bank_id i   j               
11      2   57             3
            58             2
            60             1
            61             3
            62             1
            63             2
            64             2
            66             2
            67             1
            68             1
            69             2
            70             1
            71             2
            72             3
            73             5
            74             3
            75             4
            76             2
            77             6
            78             5
            79             3
            80             1
            81             3
            82             4
            83             4
            84             5
            85             4
            86             3
            87             4
            88             2
...                      ...
12      252 228            8
            229            5
            230           12
            231           13
            232            9
            233            6
            234           10
            235           10
            236           15
            237            8
            238            8
            239            6
            240           12
            241            9
            242            6
            243           12
            244           10
            245           14
            246           15
            247           11
            248           10
            249            9
            250            6
            251            5
            252            4
            253            1
        253 249            2
            250            2
            251            1
            252            1

[125170 rows x 1 columns]

In [54]:
# Rebining by tof bin 1000us
df.groupby(['bank_id','i','j']).time_offset.resample('1000us').count()

bank_id  i    j    time_offset    
11       2    57   00:00:00.010716    1
                   00:00:00.011716    0
                   00:00:00.012716    0
                   00:00:00.013716    0
                   00:00:00.014716    0
                   00:00:00.015716    1
                   00:00:00.016716    1
              58   00:00:00.012183    1
                   00:00:00.013183    0
                   00:00:00.014183    0
                   00:00:00.015183    0
                   00:00:00.016183    0
                   00:00:00.017183    0
                   00:00:00.018183    0
                   00:00:00.019183    0
                   00:00:00.020183    1
              60   00:00:00.017958    1
              61   00:00:00.019539    2
                   00:00:00.020539    1
              62   00:00:00.018051    1
              63   00:00:00.011651    1
                   00:00:00.012651    0
                   00:00:00.013651    0
                   00:00:00.014651    0
     